---
# Introducción

En física, la energía cinética de un cuerpo es aquella energía que posee debido a su movimiento. Se define como el trabajo necesario para acelerar un cuerpo de una masa determinada desde el reposo hasta la velocidad indicada. Una vez conseguida esta energía durante la aceleración, el cuerpo mantiene su energía cinética salvo que cambie su velocidad.[1]

En el siguiente cuaderno se realizara el calculo de dicha energia para varios cuerpos, utiilzando el procesador CPU. Para ello se utilizara la formula:

 <center>Ec = 1/2 ( M * V^2)</center>

Donde Ec es enegia cinetica, M es masa del cuerpo, y V es la velocidad.

El objetivo es aprender las funciones basicas de Python[2, 3] la plataforma Colab [3, 5] y la programación secuencial.

---
# Armado del ambiente
Se debe instalar en el cuaderno módulo CUDA de Python.

In [ ]:
!pip install pycuda

---
# Desarrollo
Ejecución del codigo en GPU.

In [ ]:
# ------------------------------------------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
cantObjetos = 10000 #@param {type:"slider", min:1000, max:10000, step:1}
velocidadMaxima =   200#@param {type: "integer", min:0}
# ------------------------------------------------------------------------------

from datetime import datetime
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy

tiempo_total = datetime.now()

# ------------------------------------------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos --------------
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# ------------------------------------------------------------------------------

# Cota para la masa ------------------------------------------------------------
masaMax = 100

try: 
  if velocidadMaxima <= 0:
    raise ValueError("La velocidad maxima debe ser mayor a 0.");

  masa_cpu = numpy.random.randint(1, masaMax, size=cantObjetos)
  masa_cpu = masa_cpu.astype(numpy.int32())

  velocidad_cpu = numpy.random.uniform(1, velocidadMaxima, size=cantObjetos)
  velocidad_gpu = velocidad_cpu.astype(numpy.float32())

  energia_cpu = numpy.empty_like(velocidad_cpu)

  # CPU - reservo la memoria GPU -----------------------------------------------
  masa_gpu = cuda.mem_alloc(masa_cpu.nbytes)
  velocidad_gpu = cuda.mem_alloc(velocidad_cpu.nbytes)
  energia_gpu = cuda.mem_alloc(energia_cpu.nbytes)

  # GPU - Copio la memoria al GPU ----------------------------------------------
  cuda.memcpy_htod(masa_gpu, masa_cpu)
  cuda.memcpy_htod(velocidad_gpu, velocidad_cpu)
  cuda.memcpy_htod(energia_gpu, energia_cpu)

  # CPU - Defino la función kernel que ejecutará en GPU ------------------------
  module = SourceModule("""
  __global__ void kernel_cinetica(int n, int *mgpu, float *vgpu, float * egpu)
  {
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    if( idx < n )
    {
      egpu[idx] = 0.5 * (mgpu[idx] * pow(vgpu[idx], 2));
    }
  }
  """) 

  kernel = module.get_function("kernel_cinetica")

  tiempo_gpu = datetime.now()

  # GPU ------------------------------------------------------------------------
  dim_hilo = 256
  dim_bloque = numpy.int((cantObjetos + dim_hilo-1) / dim_hilo)

  kernel(numpy.int32(cantObjetos), masa_gpu, velocidad_gpu, energia_gpu, block=(dim_hilo, 1, 1),grid=(dim_bloque, 1,1))

  # Copio el resultado desde la memoria GPU ------------------------------------
  cuda.memcpy_dtoh(energia_cpu, energia_gpu)
  
  tiempo_gpu = datetime.now() - tiempo_gpu
  tiempo_total = datetime.now() - tiempo_total

  print("Cantidad de elementos: ", cantObjetos)
  print("Thread x: ", dim_hilo, ", Bloque x:", dim_bloque)
  print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]")
  print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]")
  

except ValueError as valerr:
  print (valerr)
except: 
  print("Houston we have a problem!")


---
# Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param cantObjetos    | Lectura de la cantidad de objetos a las cual se les aplicara el calculo.
CPU      |  @param velocidadMaxima| Lectura de la cota superior de las velocidades.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo inicial.
CPU      |  numpy.random.randint  | Inicializa el vector de las masas.
CPU      |  numpy.random.uniform  | Inicializa el vector de las velocidades de las masas.
CPU      |  numpy.empty_like      | Reservo un vector para almacenar los resultados
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel.
CPU      |  module.get_function() | Genera la función del kernel GPU.
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU.
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU a CPU.
CPU      |  datetime.             | Calculo el tiempo que se tardo en calcular las energias.
CPU      |  datetime.             | Calculo el tiempo final.
CPU      |  print()               | Informe de los resultados.


---
# Conclusión

Si bien el ejercicio realizado no presenta una gran complejidad, creo que fue necesario realizar algo simple para familiarizarme con el lenguaje de Python y como es el funcionamiento de los cuadernos de Colab; tanto para la declaración de funciones, como de variables y excepciones. Tambien me sirvio como puntapie para aprender sobre las distintas estructuras y funciones que ofrece Numpy[6]

En cuanto a los resultado, si bien el calculo realizado no requiere de mucho calculo por parte de la CPU, al comparar los valores entre la ejecición en serie y la ejecución en paralelo se ve una "ENORME" diferencia.

En promedio, con la ejecución en serie obtuve un resultado de 63.221 [ms] (tamando 10 ejecuciones con 10000 objetos). Por otra parte, con la ejecución en paralelo, el calculo de las diferentes energias se resolvió en un promedio de 0.089 [ms].

## Pasos mas relevantes
<center>1- Reservar memoria en GPU (cuda.mem_alloc).</center>
<center>2- Copiar datos en memoria GPU (cuda.memcpy_htod).</center>
<center>3- Definir la función que va a ejecutar el Kernel.</center>
<center>4- Calcular las dimensiones.</center>


---
# Bibliografía

[1] Energia Cinetica: [Wikipedia](https://es.wikipedia.org/wiki/Energ%C3%ADa_cin%C3%A9tica)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[4] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[5] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[6] Numpy: [Referencia](https://numpy.org/doc/1.16/reference/routines.random.html)